In [3]:
import numpy as np
from numpy import linalg
import sklearn 
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_svmlight_file
import math
import pandas as pd
from datetime import datetime
from scipy.spatial import distance
import re
import matplotlib.pyplot as plt
import scipy.stats as sts
from scipy.stats import ortho_group
m = ortho_group.rvs(dim=3)
from sklearn import ensemble, model_selection, metrics, tree
%matplotlib inline
from sklearn.datasets import load_digits
import time

#### Загрузим датасет и размножим его 

In [4]:
dataset = "a9a.txt"
data = load_svmlight_file(dataset)
X1, y1 = data[0].toarray(), data[1]
n1, d1 = X1.shape

X2, y2 = data[0].toarray(), data[1]
n2, d2 = X2.shape

X3, y3 = data[0].toarray(), data[1]
n3, d3 = X3.shape

In [5]:
X = np.vstack([X1, X2, X3])
y = np.concatenate([y1, y2, y3])
N, d = X.shape
print(N, d)

97683 123


#### Добавим шума

In [6]:
mu=0.0
std = 0.05 * np.std(X) # for %5 Gaussian noise
def gaussian_noise(x,mu,std):
    noise = np.random.normal(mu, std, size = x.shape)
    x_noisy = x + noise
    return x_noisy 
X = gaussian_noise(X, mu, std)

\begin{equation}
\min_{w \in \mathbb{R}^d} \frac{1}{n} \sum\limits_{i=1}^n l (g(w, x_i), y_i),
\end{equation}
где $l$ - функция потерь, $g$ - модель, $w$ - параметры модели, $\{x_i, y_i\}_{i=1}^n$ - выборка данных из векторов признаков $x_i$ и меток $y_i$.

Далее будем рассматривать линейной модель $g(w, x) = w^T x$ и квадратичную функцию потерь $l(z, y) = (z-y)^2$.
#### Рассчитаем $L, \mu$.

In [7]:
L = max(np.linalg.eigvalsh(1/N*X.T.dot(X)))
mu = min(np.linalg.eigvalsh(1/N*X.T.dot(X)))
print("L =", L)
print("mu =", mu)

L = 6.287904270918885
mu = 0.0002447896098221913


#### Построим временную симуляцию

In [8]:
def All_time(t_loc, t_comm, b, L, mu, eps, c1, c2):
    t = np.array([])
    for i in range (t_loc.shape[0]):
        t = np.append(t, t_loc[i]*b[i])
    maxim = max(t)
    #print(maxim)
    K = c1*(L/mu)**0.5*np.log(1/eps)*b[0]**-0.25
    #print(K)
    k_some = c2*(L/mu)**0.5*np.log(1/eps)
    #print(k_some)
    T = K*maxim + K*t_comm + t[0]*k_some
    #print(float(int(K*t_comm)), K, t_comm, float(int(t[0]*k_some)), k_some, t[0])
    return T

## Рассмотрим случай больших коммуникаций

In [10]:
t_comm = 1e+9
epsilon = 4.855708467588126e-09
c1 = 700.0602944751523
c2 = c1 * 1000
print(c1, c2)
alpha = c1*(L/mu)**0.5*np.log(1/epsilon)
beta = c2*(L/mu)**0.5*np.log(1/epsilon)


700.0602944751523 700060.2944751523


In [11]:
def generate_t_loc1():
    t_loc1 = np.random.uniform(3, 7, 21)
    t_loc1[0] = 1
    t_loc1 = np.array(t_loc1)
    return t_loc1
t_loc1 = generate_t_loc1()

def generate_b1(t_loc1):
    b1 = np.array([])
    b1 = np.append(b1, max(1,int(min(round((t_comm*alpha/(4*beta*t_loc1[0]))**0.8), N))))
    last = N
    for i in range (1, t_loc1.shape[0]):
        last = last - b1[-1]
        b1 = np.append(b1, round(last/(t_loc1.shape[0] - i)))
    return b1
b1 = generate_b1(t_loc1)
print(b1)

[20814.  3843.  3843.  3844.  3843.  3844.  3843.  3844.  3843.  3844.
  3843.  3844.  3843.  3844.  3843.  3844.  3843.  3844.  3843.  3844.
  3843.]


In [12]:
sum_t_loc = 0
for j in range(1, len(t_loc1)):
    sum_t_loc += t_loc1[j] ** (-1)
sum_t_loc = sum_t_loc ** (-1)

In [395]:
t1 = round(All_time(t_loc1, t_comm, b1, L, mu, epsilon, c1, c2))
print(t1)

12379243907217570


#### Рассмотрим равномерное распределение датасета

In [396]:
def generate_t_loc2():
    t_loc2 = np.random.uniform(3, 7, 21)
    t_loc2[0] = 1
    t_loc2 = np.array(t_loc2)
    return t_loc2
t_loc2 = np.copy(t_loc1)

def generate_b2(t_loc2):
    rand = np.random.uniform(1, 10, t_loc2.shape[0])
    b2 = np.array([])
    for i in range (t_loc2.shape[0]):
        b2 = np.append(b2, round(N*rand[i]/sum(rand)))
    return b2
b2 = generate_b2(t_loc2)
print(b2)

[2262. 4636. 1964. 2722. 9416. 8442. 6613. 4166. 6336. 2446. 1211. 2537.
 8180. 2466. 1491. 9019. 8187. 2093. 4055. 1942. 7500.]


#### Проведем серию из 1000 экспериментов и найдем среднее значение времени

In [397]:
t_exp = np.array([])
for i in range (1000):
    t_loc2 = generate_t_loc2()
    b2 = generate_b2(t_loc2)
    t_exp = np.append(t_exp, round(All_time(t_loc2, t_comm, b2, L, mu, epsilon, c1, c2)))
t2 = sum(t_exp)/1000
print(t2)

2.708662906157272e+16


### Итоговое ускорение

In [398]:
print("ускорили выполнение программы в  {:.3f} раза".format((t2)/t1))

ускорили выполнение программы в  2.188 раза


Общий случай

In [201]:
def F(x, aa, bb, cc):
    return (2*aa * x**0.5 + (-2)*bb * x**(-0.5) + cc * x)


b_1_0 = N/21
#0 < b_1 <= b_1_0
sum_t_loc2 = 1/(sum(1/t_loc2[i] for i in range(1, t_loc2.shape[0])))
b = - 0.5 * alpha * (N * sum_t_loc2 + t_comm)
a = - 0.5 * alpha * sum_t_loc2
c = beta * t_loc2[0]
root = (2*a**6 + 3*math.sqrt(3)*math.sqrt(4*a**3*b**3*c**6 + 27*b**4*c**8)+18*a**3*b*c**2+27*b**2*c**4)**(1/3)
xxx = a**2 / (3 * c**2) + (root)/(3*2**(1/3)*c**2) - (2**(0.33)*(-a**4-6*a*b*c**2)) / (3*c**2*root)
batch = list([0.0, 0.0, 0.0, 0.0, 0.0, 0.0])
#print(b_1_0, xxx)
if (b_1_0 < xxx ):
    batch[0] = b_1_0
else:
    batch[0] = xxx
batch[1] = F(batch[0], a, b, c)
#print(batch[1], a, b, c)
# b_1_0 < b_1 <= N 
a = 0.5 * alpha * t_loc2[0]
b = -0.5 * alpha * t_comm
c = beta * t_loc2[0]
if (N >=xxx):
    batch[2] = xxx
else:
    batch[2] = N
batch[3] = F(batch[2], a, b, c)
#print(batch[3])
if (batch[1] < batch[3]):
    batch[4] = batch[1]
    batch[5] = batch[0]
else:
    batch[4] = batch[3]
    batch[5] = batch[2]

print(batch[4], batch[5])

def All_time2(t_loc, t_comm, b, L, mu, eps, c1, c2):
    t = np.array([])
    for i in range (t_loc.shape[0]):
        t = np.append(t, t_loc[i]*b[i])
    maxim = max(t)
    #print(maxim)
    K = c1*(L/mu)**0.5*np.log(1/eps)*b[0]**-0.5
    #print(K)
    k_some = c2*(L/mu)**0.5*np.log(1/eps)
    #print(k_some)
    T = K*maxim + K*t_comm + t[0]*k_some
    return T
t_exp = np.array([])
for i in range (1000):
    t_loc2 = generate_t_loc2()
    b2 = generate_b2(t_loc2)
    t_exp = np.append(t_exp, round(All_time2(t_loc2, t_comm, b2, L, mu, epsilon, c1, c2)))
t2 = sum(t_exp)/1000
print(t2)

print("ускорили выполнение программы в  {:.1f} раза".format((t2)/batch[4]))

39530449836307.05 6124.963312683663
43753433626893.17
ускорили выполнение программы в  1.1 раза


## Рассмотрим случай малых коммуникаций

In [49]:
t_comm = 1e+6
epsilon = 4.855708467588126e-09


In [50]:
def generate_t_loc1():
    t_loc1 = np.random.uniform(3, 7, 21)
    t_loc1[0] = 1
    t_loc1 = np.array(t_loc1)
    return t_loc1
t_loc1 = generate_t_loc1()

def generate_b1(t_loc1):
    b1 = np.array([])
    S = (sum(1/t_loc1[i] for i in range(1, t_loc1.shape[0])))**(-1)
    b1 = np.append(b1, max(1, round(N*S/(t_loc1[0] + S))))
    last = N
    for i in range (1, t_loc1.shape[0]):
        last -=  b1[-1]
        S = (sum(1/t_loc1[k] for k in range(i, t_loc1.shape[0])))**(-1)
        b1 = np.append(b1, round(last/t_loc1[i]*S))
    
    return b1
b1 = generate_b1(t_loc1)
print(b1)

[18584.  4240.  3222.  3223.  2657.  4325.  3985.  3035.  4428.  2689.
  4422.  2745.  4444.  6003.  3732.  3555.  3292.  4801.  3137.  5984.
  5180.]


#### Проведем серию из 1000 экспериментов и найдем среднее значение времени

In [51]:
t_exp = np.array([])
for i in range (1000):
    t_loc1 = generate_t_loc1()
    b1 = generate_b1(t_loc1)
    t_exp = np.append(t_exp, round(All_time(t_loc1, t_comm, b1, L, mu, 0.1, c1, c2)))
t1 = sum(t_exp)/1000
print(t1)

8.389709017556795e+16


In [52]:
def generate_t_loc2():
    t_loc2 = np.random.randint(3, 7, 21)
    t_loc2[0] = 1
    t_loc2 = np.array(t_loc2)
    return t_loc2
t_loc2 = np.copy(t_loc1)

def generate_b2(t_loc2):
    rand = np.random.uniform(0, N, t_loc2.shape[0])
    rand = np.sort(rand)
    for j in rand:
        j = int(j)
    rand = np.append(rand, N)
    #print(rand)
    b2 = np.array([])
    b2 = np.append(b2, max(1, round(rand[1] - rand[0])))
    for i in range (1, t_loc2.shape[0]):
        b2 = np.append(b2, round(rand[i+1] - rand[i]))
    return b2
b2 = generate_b2(t_loc2)
print(b2)

[2.6150e+03 4.7940e+03 3.9100e+02 2.4630e+03 3.8590e+03 6.0100e+02
 1.2305e+04 2.2740e+03 1.7140e+03 3.3660e+03 1.7220e+03 2.0810e+03
 1.0900e+02 3.5831e+04 4.4800e+02 7.0000e+00 2.8100e+03 2.8170e+03
 8.4660e+03 3.7500e+03 2.7310e+03]


#### Проведем серию из 1000 экспериментов и найдем среднее значение времени

In [53]:
t_exp = np.array([])
for i in range (1000):
    t_loc2 = generate_t_loc2()
    b2 = generate_b2(t_loc2)
    t_exp = np.append(t_exp, round(All_time(t_loc2, t_comm, b2, L, mu, epsilon, c1, c2)))
t2 = sum(t_exp)/1000
print(t2)

1.1017824505095891e+18


### Итоговое ускорение

In [54]:
print("ускорили выполнение программы в  {:.5f} раза".format((t2)/t1))

ускорили выполнение программы в  13.13255 раза


Общий случай: 

In [55]:
def F(x, aa, bb, cc):
    return (2*aa * x**0.5 + (-2)*bb * x**(-0.5) + cc * x)


b_1_0 = N/5
#0 < b_1 <= b_1_0
sum_t_loc2 = 1/(sum(1/t_loc2[i] for i in range(1, t_loc2.shape[0])))
b = - 0.5 * alpha * (N * sum_t_loc2 + t_comm)
a = - 0.5 * alpha * sum_t_loc2
c = beta * t_loc2[0]
root = (2*a**6 + 3*math.sqrt(3)*math.sqrt(4*a**3*b**3*c**6 + 27*b**4*c**8)+18*a**3*b*c**2+27*b**2*c**4)**(1/3)
xxx = a**2 / (3 * c**2) + (root)/(3*2**(1/3)*c**2) - (2**(0.33)*(-a**4-6*a*b*c**2)) / (3*c**2*root)
batch = list([0.0, 0.0, 0.0, 0.0, 0.0, 0.0])
print(b_1_0, xxx)
if (b_1_0 < xxx ):
    batch[0] = round(b_1_0)
else:
    batch[0] = round(xxx)
batch[1] = F(batch[0], a, b, c)
print(batch[1], a, b, c)
# b_1_0 < b_1 <= N 
a = 0.5 * alpha * t_loc2[0]
b = -0.5 * alpha * t_comm
c = beta * t_loc2[0]
if (N < xxx):
    batch[2] = round(N)
elif (xxx < b_1_0):
    batch[2] = round(b_1_0)
else:
    batch[2] = round(xxx)
batch[3] = F(batch[2], a, b, c)
print(batch[3])
if (batch[1] < batch[3]):
    batch[4] = batch[1]
    batch[5] = batch[0]
else:
    batch[4] = batch[3]
    batch[5] = batch[2]

print(batch[4], batch[5])

19536.6 63.84840586474767
411404288350.4705 -223734.62345606327 -1095781261812.1624 2147852385.178207
41978258783724.14
411404288350.4705 64


In [221]:
def All_time2(t_loc, t_comm, b, L, mu, eps, c1, c2):
    t = np.array([])
    for i in range (t_loc.shape[0]):
        t = np.append(t, t_loc[i]*b[i])
    maxim = max(t)
    #print(maxim)
    K = c1*(L/mu)**0.5*np.log(1/eps)*b[0]**-0.5
    #print(K)
    k_some = c2*(L/mu)**0.5*np.log(1/eps)
    #print(k_some)
    T = K*maxim + K*t_comm + t[0]*k_some
    return T

t_exp = np.array([])
for i in range (1000):
    t_loc2 = generate_t_loc2()
    b2 = generate_b2(t_loc2)
    t_exp = np.append(t_exp, round(All_time2(t_loc2, t_comm, b2, L, mu, epsilon, c1, c2)))
t2 = sum(t_exp)/1000
print(t2)

t_exp = np.array([])
for i in range (1000):
    t_loc2 = generate_t_loc2()
    b2 = np.array([5])
    last = N - b2[0]
    for i in range (1, t_loc2.shape[0]):
        last = last - b2[-1]
        b2 = np.append(b2, round(last/(t_loc2.shape[0] - i)))
    t_exp = np.append(t_exp, round(All_time2(t_loc2, t_comm, b2, L, mu, epsilon, c1, c2)))
t222 = sum(t_exp)/1000
print(t222)

9735273891553.12
38941137336.426


In [223]:
print("ускорили выполнение программы в  {:.1f} раза".format((t2)/batch[4]))
#print("ускорили выполнение программы в  {:.1f} раза".format((t2)/t222))

ускорили выполнение программы в  904.6 раза


Умножение матриц

In [215]:
def multiplication(XX,YY):
    print(XX.shape)
    XX_len = XX.shape[0]
    YY_len = XX.shape[1]
    result = np.zeros((XX_len, XX_len))
    begin_time = time.time()
    for i in range(XX_len):
     # iterate through columns of Y
        for j in range(XX_len):
     # iterate through rows of Y
            for k in range(YY_len):
                result[i][j] += XX[i][k] * YY[k][j]
    return (time.time() - begin_time)

def generate_matrix(yyy, xxx):
    return np.array([[np.random.uniform (-100, 100) for i in range (xxx)] for j in range (yyy)])

In [216]:
Sizes = [40*k for k in range(12)]
#print(Sizes)
timelist = []
for j in Sizes:
    Matrix1 = generate_matrix(123, j)
    timelist.append(multiplication(Matrix1.T, Matrix1))

(0, 123)
(40, 123)
(80, 123)
(120, 123)


KeyboardInterrupt: 

In [ ]:
Sizes = [40*k for k in range(1, 12)]
#print(Sizes)
timelist2 = []
for j in Sizes:
    Matrix1 = generate_matrix(j, 123)
    timelist2.append(multiplication(Matrix1.T, Matrix1))

In [ ]:
plt.figure(figsize=(12, 8))

plt.plot([40*k for k in range(1, 12)], timelist[1:], linewidth=4, label = 'd = 40 to 440')
plt.plot([40*k for k in range(1, 12)], timelist2, linewidth=4, label = 'N = 40 to 440')

plt.legend(loc="upper right", fontsize=15)
plt.xlabel(r"N, d", fontsize=30)
plt.ylabel("time", fontsize=30)
plt.title(r"Зависимость времени работы от размера", fontsize=30)
plt.xticks(fontsize=25)
plt.yticks(fontsize=25)
plt.grid()

In [ ]:
Sizes = [200*k for k in range(1, 5)]
#print(Sizes)
timelist3 = []
for j in Sizes:
    Matrix1 = generate_matrix(123, j)
    timelist3.append(multiplication(Matrix1.T, Matrix1))

In [ ]:
Sizes = [200*k for k in range(1, 5)]
#print(Sizes)
timelist4 = []
for j in Sizes:
    Matrix1 = generate_matrix(j, 123)
    timelist4.append(multiplication(Matrix1.T, Matrix1))

In [ ]:
plt.figure(figsize=(12, 8))

plt.plot([200*k for k in range(1, 5)], timelist3, linewidth=4, label = 'd = 200 to 800')
plt.plot([200*k for k in range(1, 5)], timelist4, linewidth=4, label = 'N = 200 to 800')

plt.legend(loc="upper right", fontsize=15)
plt.xlabel(r"N, d", fontsize=30)
plt.ylabel("time", fontsize=30)
plt.title(r"Зависимость времени работы от размера", fontsize=30)
plt.xticks(fontsize=25)
plt.yticks(fontsize=25)
plt.grid()

In [19]:
# Поиск минимума функции
from scipy.optimize import minimize_scalar
from math import ceil, floor

def func1(x):
        return a1*x**(3/4) + b1*x**(-1/4) + c1*x
    def func2(x):
        return a2*x**(3/4) + b2*x**(-1/4) + c2*x

def minimum_finding(t_comm):
    t_comm = 1e-9
    b_1_0 = N*(sum_t_loc)/(sum_t_loc + t_loc1[0])
    #0 < b_1 <= b_1_0

    a1 = - alpha * sum_t_loc
    b1 = alpha * (sum_t_loc * N + t_comm)
    c1 = beta * t_loc1[0]

    a2 = alpha * t_loc1[0]
    b2 = alpha * (t_comm)
    c2 = beta * t_loc1[0]

    

    res1 = minimize_scalar(func1, bounds=(1,floor(b_1_0)), method='bounded')
    res2 = minimize_scalar(func1, bounds=(ceil(b_1_0),N), method='bounded')

    #print(b_1_0)
    
    res11 = floor(res1.x)
    res12 = ceil(res1.x)
    res21 = floor(res2.x)
    res22 = ceil(res2.x)
    fres11 = func1(res11)
    fres12 = func1(res12)
    fres21 = func2(res21)
    fres22 = func2(res22)

    if (fres11 < fres12):
        result1 = [res11, fres11]
    else:
        result1 = [res12, fres12]


    if (fres21 < fres22):
        result2 = [res21, fres21]
    else:
        result2 = [res22, fres22]
    
    if (result1[1] < result2[1]):
        result = [result1[0], result1[1]]
    else:
        result = [result2[0], result2[1]]

    return result[0]

19649.443056980635
Минимальное значение функции 1 = 45922074190.612335
x1, где достигается минимум = 4.276691212092229
Минимальное значение функции 2 = 42208865154932.78
x2, где достигается минимум = 19650.00055177286


In [29]:
res11 = floor(res1.x)
res12 = ceil(res1.x)
res21 = floor(res2.x)
res22 = ceil(res2.x)
fres11 = func1(res11)
fres12 = func1(res12)
fres21 = func2(res21)
fres22 = func2(res22)


if (fres11 < fres12):
    result1 = [res11, fres11]
else:
    result1 = [res12, fres12]

print(result1)

if (fres21 < fres22):
    result2 = [res21, fres21]
else:
    result2 = [res22, fres22]
    
print(result2)

[4, 45947333199.321465]
[19650, 42208864096333.52]


In [30]:
if (result1[1] < result2[1]):
    result = [result1[0], result1[1]]
else:
    result = [result2[0], result2[1]]

print(result)

[4, 45947333199.321465]
